<a href="https://colab.research.google.com/github/SophiaKleistKarlson/CDS-final-project/blob/main/Image_preprocessing_script.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Preprocessing script for images**

The images are first uploaded to google colaboratory as a zipfile. A .csv file created in the R preprocessing script is also uploaded, containing unique names for each image and their path. 

In [ ]:
# load necessary modules
import glob
import re
import os
from google.colab import files

# define path
path = "/content/"
os.chdir(path)
    
# create folder for all images to be put in
if not os.path.exists('data'):
    os.makedirs('data')

# define new path where we will put the zip file with all images
new_path = "/content/data/"
os.chdir(new_path)
print(new_path)
    
# import zipfile 'transmission_VY53sLXHyJ6H_0G5z021ZWZdA.zip' (containing all the images) into the data folder
uploaded = files.upload()

# also import 'Drawing_ID.csv' containing unique drawing names and paths
uploaded = files.upload()

# unzip the zipfile
!unzip 'transmission_VY53sLXHyJ6H_0G5z021ZWZdA.zip' #"my_archive.zip"

Now, assign unique names to images using the Drawing_ID.csv file and put them all in one folder.

In [ ]:
# import the necessary packages
import pandas as pd
import cv2
import os

# set path
path = '/content/'
os.chdir(path)

# create folder for all images to be put in
if not os.path.exists('all_drawings'):
    os.makedirs('all_drawings')

# Import the csv file with image paths and unique names
Drawing_IDs = pd.read_csv('data/Drawing_IDs.csv')

# make the image path column to a list
image_path_list = Drawing_IDs[["image_path"]]
image_path_list = image_path_list["image_path"].tolist()

# make the image ID column to a list
image_ID_list = Drawing_IDs[["Drawing_ID"]]
image_ID_list = image_ID_list["Drawing_ID"].tolist()

# Loop that reads the image paths and saves them into the all_drawings folder with their unique names
for i in range(len(image_ID_list)):
    drawing = cv2.imread(image_path_list[i])
    drawing_ID = "all_drawings/" + image_ID_list[i] + ".png"
    cv2.imwrite(drawing_ID, drawing)


All drawings are now resized to 400x400 px.

In [ ]:
# load necessary modules
import cv2
import glob
import re
import os


# define path where all drawings are
path = "/content/"
os.chdir(path)
print(path)

# make a list of all the drawings
img_list = glob.glob(path + "all_drawings/*.png")
img_list.sort() # sort the list alphabetically

print(len(img_list))

# make empty list to be filled with filenames
id = []

# make folder for the resized images
if not os.path.exists('resized'):#'all_resized_drawings'
    os.makedirs('resized')

# loop that resizes each image to 400x400 px
for i in range(len(img_list)):
  id.append(re.findall('/content/all_drawings/(Chain_\d+_Gen_\d+_Cond_\d+_Source_\d+).png', img_list[i])[0])
  src = cv2.imread(img_list[i], cv2.IMREAD_UNCHANGED)

  # set a new height and width in pixels
  new_height = 400
  new_width = 400

  # dsize
  dsize = (new_width, new_height)

  # resize image
  output = cv2.resize(src, dsize)
  
  # write the resized image and put it in the all_resized_drawings folder
  cv2.imwrite('/content/resized/{}.png'.format(id[i]), output) #'C:/Users/Sophia/Documents/Social Transmission Study/Analysis of drawings/data/all_drawings/all_resized_drawings/{}.png'



The resized images are blurred using cv.blur(), using the convolution function, and then converted to jpeg2000 format. The unblurred images are also converted into jpeg2000 format. The file sizes these two batches of .jp2 images are calculated, put into a dataframe and saved as a csv. This csv as well as the resized images, blurred images and jpeg2000 files are zipped and downloaded.

In [ ]:
# import the necessary packages
import PIL
from PIL import Image
from pathlib import Path
import numpy as np
import pandas as pd
import glob
import os
import re
import cv2
import cv2 as cv
from google.colab import files
from zipfile import ZipFile

# Define function that blurs images, converts them and the original into .jp2 format, reads filesizes and puts it all into a pandas dataframe
def complexity_comparison_function(img_list):
  
  # make directory to put jp2 files
  if not os.path.exists('jpeg_compressed_files_comparison'):
    os.makedirs('jpeg_compressed_files_comparison')

  # make directory to put blurred png files
  if not os.path.exists('blurred_images'):
    os.makedirs('blurred_images')
    
  # prepare dataframe
  columns = ['Drawing_ID', 'Complexity_original', 'Complexity_convolution']
  index = np.arange(0)
  Complexity_comparison_data = pd.DataFrame(columns=columns, index = index)

  # list for id's
  id = []
  
  # convert png to jp2
  for i in range(len(img_list)):
    id.append(re.findall('/content/resized/(Chain_\d+_Gen_\d+_Cond_\d+_Source_\d+).png', img_list[i])[0]) 
    
    #complexity of original:
    img_original = Image.open(img_list[i])
    img_original.convert("RGBA").save("jpeg_compressed_files_comparison/Original_{}.jp2".format(id[i]), 'JPEG2000', quality_mode='dB', quality_layers=[80])
    
    # read image for blurring
    img_blur = cv.imread(img_list[i])

    # convolution
    kernel = np.ones((5,5),np.float32)/25
    dst = cv.filter2D(img_blur,-1,kernel)
    cv2.imwrite('/content/blurred_images/Convolution_{}.png'.format(id[i]), dst)

    # jpeg2000 convertion of blurred images
    img_convolution = Image.open("/content/blurred_images/Convolution_{}.png".format(id[i]))
    img_convolution.convert("RGBA").save("jpeg_compressed_files_comparison/Convolution_{}.jp2".format(id[i]), 'JPEG2000', quality_mode='dB', quality_layers=[80])

  # list of original jp2 files
  img_jp2_original = glob.glob("jpeg_compressed_files_comparison/Original_*.jp2")
  img_jp2_original.sort() #important! Else the IDs and complexity scores won't match

  # list of convolution jp2 files
  img_jp2_convolution = glob.glob("jpeg_compressed_files_comparison/Convolution_*.jp2")
  img_jp2_convolution.sort()

  # save size and ID to dataframe
  for i in range(len(img_jp2_original)): 
    size_original = (Path(img_jp2_original[i]).stat().st_size)
    size_convolution = (Path(img_jp2_convolution[i]).stat().st_size)

    Complexity_comparison_data = Complexity_comparison_data.append({
      'Drawing_ID': id[i],
      'Complexity_original': size_original, 
      'Complexity_convolution': size_convolution
    }, ignore_index=True)

  return Complexity_comparison_data


# define path where all drawings are
path = "/content/resized/" 
images = glob.glob(path + "*.png")
print(len(images))
images.sort() #important! Else the IDs and complexity scores won't match

# run function on all drawings
complexity_comparison_data = complexity_comparison_function(images)

# print complexity scores and ID's
print(complexity_comparison_data)

# prepare logfilename and save
logfilename = "complexity_comparison.csv"
complexity_comparison_data.to_csv(logfilename)

# prepare zipfile
zipObj = ZipFile('image_processing.zip', 'w')

# add complexity_comparison.csv to the zipfile
zipObj.write('complexity_comparison.csv')

# prepare lists of resized and blurred images and jp2 files
resized = glob.glob('resized/' + "*.png")
blurred = glob.glob('blurred_images/' + "*.png")
jpeg2000 = glob.glob('jpeg_compressed_files_comparison/' + "*.jp2")

# loop over the resized and blurred images and add them to the zipfile
for i in range(len(blurred)):
  # Add multiple files to the zipfile
  zipObj.write(resized[i])
  zipObj.write(blurred[i])

# loop over the jp2 files and add them to the zipfile
for i in range(len(jpeg2000)):
  zipObj.write(jpeg2000[i])

# close the zipfile
zipObj.close()

# download zipfile
files.download("image_processing.zip") 

Calculating complexity of resized source images:

**NB:** First, I manually created a folder called "resized_source" and uploaded the 12 resized source images here.



In [ ]:
# import the necessary packages
import PIL
from PIL import Image
from pathlib import Path
import numpy as np
import pandas as pd
import glob
import os
import re
import cv2
import cv2 as cv
from google.colab import files
from zipfile import ZipFile

# Define function
def complexity_comparison_function(img_list):
  
  # make directory to put jp2 files
  if not os.path.exists('jpeg_compressed_files_comparison_source'):
    os.makedirs('jpeg_compressed_files_comparison_source')

  # make directory to put blurred png files
  if not os.path.exists('blurred_source_images'):
    os.makedirs('blurred_source_images')
    
  # prepare dataframe
  columns = ['Drawing_ID', 'Complexity_original', 'Complexity_convolution']
  index = np.arange(0)
  Complexity_comparison_data = pd.DataFrame(columns=columns, index = index)

  # list for id's
  id = []
  
  # convert png to jp2
  for i in range(len(img_list)):
    id.append(re.findall('/content/resized_source/(stim_\d+).png', img_list[i])[0]) 
    
    #complexity of original:
    img_original = Image.open(img_list[i])
    img_original.convert("RGBA").save("jpeg_compressed_files_comparison_source/Original_{}.jp2".format(id[i]), 'JPEG2000', quality_mode='dB', quality_layers=[80])
    
    # read image for blurring
    img_blur = cv.imread(img_list[i])

    # convolution
    kernel = np.ones((5,5),np.float32)/25
    dst = cv.filter2D(img_blur,-1,kernel)
    cv2.imwrite('/content/blurred_source_images/Convolution_{}.png'.format(id[i]), dst)

    # jpeg2000 convertion of blurred images
    img_convolution = Image.open("/content/blurred_source_images/Convolution_{}.png".format(id[i]))
    img_convolution.convert("RGBA").save("jpeg_compressed_files_comparison_source/Convolution_{}.jp2".format(id[i]), 'JPEG2000', quality_mode='dB', quality_layers=[80])

  # list of original jp2 files
  img_jp2_original = glob.glob("jpeg_compressed_files_comparison_source/Original_*.jp2")
  img_jp2_original.sort() #important! Else the IDs and complexity scores won't match

  # list of convolution jp2 files
  img_jp2_convolution = glob.glob("jpeg_compressed_files_comparison_source/Convolution_*.jp2")
  img_jp2_convolution.sort()

  # save size and ID to dataframe
  for i in range(len(img_jp2_original)): 
    size_original = (Path(img_jp2_original[i]).stat().st_size)
    size_convolution = (Path(img_jp2_convolution[i]).stat().st_size)

    Complexity_comparison_data = Complexity_comparison_data.append({
      'Drawing_ID': id[i],
      'Complexity_original': size_original, 
      'Complexity_convolution': size_convolution
    }, ignore_index=True)

  return Complexity_comparison_data


# define path where all drawings are
path = "/content/resized_source/" 
images = glob.glob(path + "*.png")
print(len(images))
images.sort() #important! Else the IDs and complexity scores won't match

# run function on all drawings
complexity_comparison_data = complexity_comparison_function(images)

# print complexity scores and ID's
print(complexity_comparison_data)

# prepare logfilename and save
logfilename = "complexity_comparison_source.csv"
complexity_comparison_data.to_csv(logfilename)

# prepare zipfile
zipObj = ZipFile('source_image_processing.zip', 'w')

# add complexity_comparison.csv to the zipfile
zipObj.write('complexity_comparison_source.csv')

# prepare lists of resized and blurred images and jp2 files
resized = glob.glob('resized_source/' + "*.png")
blurred = glob.glob('blurred_source_images/' + "*.png")
jpeg2000 = glob.glob('jpeg_compressed_files_comparison_source/' + "*.jp2")

# loop over the resized and blurred images and add them to the zipfile
for i in range(len(blurred)):
  # Add multiple files to the zipfile
  zipObj.write(resized[i])
  zipObj.write(blurred[i])

# loop over the jp2 files and add them to the zipfile
for i in range(len(jpeg2000)):
  zipObj.write(jpeg2000[i])

# close the zipfile
zipObj.close()

# download zipfile
files.download("source_image_processing.zip") 